In [38]:
#!pip install torch torchvision
#!pip install -U ultralytics
#!pip install opencv-python
#!pip install pandas

In [2]:
import torch
import torchvision
import ultralytics
import cv2
import os
import csv
import pandas as pd
import yaml
from ultralytics import YOLO
import numpy as np
import glob

!unzip data-2.zip
!mv data-2 datasets
!mv datasets/training_images datasets/images 

Archive:  data-2.zip
   creating: data-2/
  inflating: __MACOSX/._data-2       
   creating: data-2/testing_images/
  inflating: __MACOSX/data-2/._testing_images  
  inflating: data-2/train_solution_bounding_boxes (1).csv  
  inflating: __MACOSX/data-2/._train_solution_bounding_boxes (1).csv  
   creating: data-2/training_images/
  inflating: __MACOSX/data-2/._training_images  
  inflating: data-2/sample_submission.csv  
  inflating: __MACOSX/data-2/._sample_submission.csv  
  inflating: data-2/testing_images/vid_5_30040.jpg  
  inflating: __MACOSX/data-2/testing_images/._vid_5_30040.jpg  
  inflating: data-2/testing_images/vid_5_27500.jpg  
  inflating: __MACOSX/data-2/testing_images/._vid_5_27500.jpg  
  inflating: data-2/testing_images/vid_5_28380.jpg  
  inflating: __MACOSX/data-2/testing_images/._vid_5_28380.jpg  
  inflating: data-2/testing_images/vid_5_26420.jpg  
  inflating: __MACOSX/data-2/testing_images/._vid_5_26420.jpg  
  inflating: data-2/testing_images/vid_5_31160.jpg  

In [41]:
# Input CSV file and output directory
csv_file = 'datasets/train_solution_bounding_boxes (1).csv'
output_dir = 'datasets/labels/'
Class_car = 0
img_height = 380
img_widht = 676
# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Read the CSV file and create a .txt file for each row
with open(csv_file, mode='r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        # Extract information from the CSV row
        image = row['image']

        xmin = float(row['xmin'])
        ymin = float(row['ymin'])
        xmax = float(row['xmax'])
        ymax = float(row['ymax'])

        x_center = (xmax+xmin)/(2*img_widht)
        y_center = (ymax+ymin)/(2*img_height)
        width = (xmax-xmin)/img_widht
        height = (ymax-ymin)/img_height    

        # Create a .txt file with the same name as the image
        txt_filename = os.path.join(output_dir, os.path.splitext(image)[0] + '.txt')

        # Write the bounding box coordinates to the .txt file
        with open(txt_filename, mode='w') as txt_file:
            txt_file.write(f"{Class_car} {x_center} {y_center} {width} {height}")

print("Conversion completed.")

Conversion completed.


In [ ]:
list_img = sorted(os.listdir('datasets/images/'))
list_labels = sorted(os.listdir('datasets/labels/'))

for i, element in enumerate(list_labels):
    element = element.split('.')[0]
    list_labels[i] = element

for i, element in enumerate(list_img):
    element = element.split('.')[0]
    list_img[i] = element


data_list = np.isin(list_img, list_labels)
data_list = np.array(list_img)[data_list]

train_data = data_list[int(len(data_list)*0.2):]
val_data = data_list[:int(len(data_list)*0.2)]




In [ ]:
import os
import shutil

image_dir = 'datasets/images/'
label_dir = 'datasets/labels/'


!mkdir datasets/images/train/
!mkdir datasets/images/val/
!mkdir datasets/images/unlabelled/

!mkdir datasets/labels/train/
!mkdir datasets/labels/val/


for img_file in train_data:
    
    src_img_path = os.path.join(image_dir, img_file + '.jpg')
    dest_img_path = os.path.join(image_dir + 'train/', img_file + '.jpg')
    try:
        shutil.move(src_img_path, dest_img_path)
    except: print(img_file, 'ne correspond pas')

    src_label_path = os.path.join(label_dir, img_file + '.txt')
    dest_label_path = os.path.join(label_dir + 'train/', img_file + '.txt')
    try:
        shutil.move(src_label_path, dest_label_path)
    except: print(img_file, 'ne correspond pas')

for img_file in val_data:
    
    src_img_path = os.path.join(image_dir, img_file + '.jpg')
    dest_img_path = os.path.join(image_dir + 'val/', img_file + '.jpg')
    try:
       shutil.move(src_img_path, dest_img_path)
    except: print(img_file, 'ne correspond pas')

    src_label_path = os.path.join(label_dir, img_file + '.txt')
    dest_label_path = os.path.join(label_dir + 'val/', img_file + '.txt')
    try:
        shutil.move(src_label_path, dest_label_path)
    except: print(img_file, 'ne correspond pas')

list_unlabelled = sorted(glob.glob('datasets/images/*.jpg'))

for i, element in enumerate(list_unlabelled):
    element = os.path.basename(element)
    list_unlabelled[i] = element

for img_file in list_unlabelled:

    src_img_path = os.path.join(image_dir , img_file)
    dest_img_path = os.path.join(image_dir + 'unlabelled/', img_file)
    shutil.move(src_img_path, dest_img_path)

In [ ]:
#validation_img = list_img[:int(len(list_img)*0.2)]
validation_labels = list_labels[:int(len(list_labels)*0.2)]

#len(validation_img)
len(validation_labels)

In [ ]:
yolo_format=dict(path="",
                 train="images/train",
                 val="images/val",
                 nc=1,
                 names={0:"car"})
             
with open('yolo.yaml', 'w') as outfile:
    yaml.dump(yolo_format, outfile, default_flow_style=False)

In [ ]:
model=YOLO('yolov5su.pt')
model.train(data="yolo.yaml",epochs=5,patience=5,batch=8,
                    lr0=0.005)

In [1]:
model = YOLO('yolov8m_trained.pt')

NameError: name 'YOLO' is not defined

In [ ]:
with torch.no_grad():
    results=model.predict(source='datasets/testing_images',conf=0.50,iou=0.75)

In [ ]:
!mkdir prediction/